# Chapter 16: Data formats I (CSV and TSV)

You probably have heard of (or are already quite familiar with) different data formats, such as plain text, tables (CSV/TSV), XML, JSON and RDF. These formats are simply the result of agreements that were made between people on how to organize and store data. Some of these formats, such as XML and RDF, have a high degree of structure, whereas plain text is a typical example of unstructured data. Structuring data according to predefined specifications allows information in the data to be easily ordered and processed by machines. You can compare highly structured data with a perfectly organized filing cabinet where everything is identified, labeled and easy to access. 

This notebook introduces tabular formats: CSV/TSV. 

**At the end of this chapter, you will be able to:**
* read CSV/TSV data
* manipulate CSV/TSV data
* write CSV/TSV data

**If you want to learn more about these topics, you might find the following links useful:**
* [Tutorial: Reading and Manipulating CSV Files](https://newcircle.com/s/post/1572/python_for_beginners_reading_and_manipulating_csv_files)

If you have **questions** about this chapter, please send an email to 
**python1819vu@googlegroups.com**. 

## 1. Introduction to CSV and TSV (tables)
The **table** is probably one of the most common and intuitive data formats. Undoubtedly, you have already worked with tabular data in Excel, Numbers or Google Sheets. A table represents a set of data points as a series of rows, with a column for each of the data points' properties. In other words: a table has vertical **columns** (often identifiable by name) and horizontal **rows**, and the **cells** are the unit where a row and column intersect. Tabular data can be encoded as **CSV (comma-separated values) or TSV (tab-separated values)**. CSV and TSV files are simply plain text files in which each line represents a row and the columns are separated by a comma (for CSV) or a tab character (for TSV).

You can find examples of CSV files in the folder `../Data/baby_names/names_by_state`. If you like, open them in a text editor (e.g. [Atom](https://atom.io/), [BBEdit](https://www.barebones.com/products/bbedit/download.html) or [Notepad++](https://notepad-plus-plus.org)) or Excel (convert text to columns by using the comma as delimiter) to see their content. For example, these are the first 10 rows of the csv file `AK.TXT`:

CSV            |  
:-------------------------:|:-------------------------:
![box](./images/csv.png)  |

A TSV file would look like this:

TSV            |  
:-------------------------:|:-------------------------:
![box](./images/tsv.png)  |


The file `AK.TXT` contains a list of names given to children in the state Alaska from 1910 to 2015 with their frequency. Each line in this file has five elements, which are separated by commas: 

- the state abbreviation (AK for Alaska)
- gender (F/M)
- year
- name
- frequency of that name in the given year and state 

Below, we will work towards representing this data in Python as a **list of lists** (i.e. nested list), or as a **list of dicts**. In both cases, the elements of the (first) list represent the complete rows. The individual rows, then, can be either represented as a list (without column names) or as a dictionary (with column names).

LIST OF LISTS            |  LIST OF DICTS
:-------------------------:|:-------------------------:
![box](./images/list_of_lists.png)  |  ![box](./images/list_of_dicts.png)

## 2. Reading CSV files

Because CSV/TSV files are essentially text files, we can open and read them in the same way as we have seen before:

In [ ]:
# Read the file and print its content 
filename = "../Data/baby_names/names_by_state/AK.TXT"
with open(filename, "r") as csvfile:
    content = csvfile.read()
    print(content)

Please also have a close look at the internal representation of the file. Do you see how the columns are separated by commas, and the rows by newline characters `\n`?

In [ ]:
print(repr(content))

### 2.1 Reading rows as lists
Now, let's see how we can get to the 'list of lists' representation. We can do that by iterating over each line of this file (as we have seen before), and then split each row into columns using the `split()` method:

In [ ]:
# Read the file and get all lines
filename = "../Data/baby_names/names_by_state/AK.TXT"
with open(filename, "r") as csvfile:
    csv_data = []
    for row in csvfile:
        row = row.strip("\n")      # remove all newlines
        columns = row.split(",")   # split the line into columns
        csv_data.append(columns) 

# Print only first 10 rows
print(csv_data[:10])
print()

# Iterate over first 10 rows
for row in csv_data[:10]:
    print(row)

It worked! The variable `csv_data` now contains a list of all rows in the file. Now we can easily work with the data by using the indices of the lists to access cells. Have a look at the following examples:

In [ ]:
# Example: print all information of the first 10 rows
for row in csv_data[:10]:
    state = row[0]
    gender = row[1]
    year = row[2]
    name = row[3]
    frequency = row[4]
    print(state, gender, year, name, frequency)

In [ ]:
# Example: print all names given in 1912
for row in csv_data:
    year = row[2]
    name = row[3]
    if year == "1912":
        print(name)

### 2.2 Reading rows as dicts

We can also create a 'list of dicts'. We do this by first creating a dictionary for each row, and appending that dictionary to the list of rows. Do you see that we also have already converted the numerical values to `int`?

In [ ]:
# Read the file and get all lines
filename = "../Data/baby_names/names_by_state/AK.TXT"
with open(filename, "r") as csvfile:
    csv_data = []
    for row in csvfile:
        row = row.strip("\n")      # remove all newlines
        columns = row.split(",")   # split the line into columns
        
        # Create a dictionary and add to list
        dict_row = {"state": columns[0],
                    "gender": columns[1],
                    "year": int(columns[2]),
                    "name": columns[3],
                    "frequency": int(columns[4])}
        csv_data.append(dict_row)

# First 10 rows
print(csv_data[:10])
print()

# Iterate over first 10 rows 
for row in csv_data[:10]:
    print(row)

Again, we can now easily work with the data, but now we use the *names* of the columns instead of indices to access the cells. This can make the code more readable. Have a look at the following examples:

In [ ]:
# Example: print all information of the first 5 rows
for row in csv_data[:5]:
    for column_name, column_value in row.items():
        print(column_name, "=", column_value)
    print()

In [ ]:
# Example: print all names given in 1912
for row in csv_data:
    if row["year"] == "1912":
        print(row["name"])

It does not really matter whether you choose for a 'list of lists' or a 'list of dicts'. Just use the one that you prefer.

## 3. Writing CSV files

Let's say now we have a table in Python stored as a 'list of lists' or as a 'list of dicts' and we want to store our result in a CSV file. This is basically the inverse process of reading a CSV file. 

### 3.1 Writing rows as lists

In order to write a list of lists as a CSV file, we need to iterate over the rows and make a string out of them. Remember that we can concatenate strings in a list with any separator with the `join()` method:

In [ ]:
a_list = ["John", "john@example.nl", "555-1234"]
a_string = ",".join(a_list)
print(a_string)

In addition, we should use the newline character `\n` to write each row on a line.

In [ ]:
# Create list of lists
address_book = [
    ["John", "john@example.nl", "555-1234"],
    ["William", "william@example.nl", "555-5678"],
    ["Jane", "jane@example.nl", "555-7777"]
]

# Write the list of lists to a CSV file
outfilename = "../Data/address_book.csv"
with open(outfilename, "w") as outfile:
    for row in address_book:
        line = ",".join(row) + '\n'
        outfile.write(line)

### 3.2 Writing rows as dicts

In order to write a list of dicts to a CSV file, we need to first get all the values in each dictionary. The rest works exactly the same. In the following code, we use the tab separator `\t` and save it with the `.tsv` extension.

In [ ]:
# Create list of dicts
address_book = [
    {"name":"John", "e-mail":"john@example.nl", "phone":"555-1234"},
    {"name":"William", "e-mail":"william@example.nl", "phone":"555-5678"},
    {"name":"Jane", "e-mail":"john@example.nl", "phone":"555-7777"}
]

# Write the list of dicts to a TSV file
outfilename = "../Data/csv_data/address_book.tsv"
with open(outfilename, "w") as outfile:    
    
    # Write the rows using the values of the dictionaries
    for row in address_book:
        column_values = row.values()
        line = "\t".join(column_values) + '\n'
        outfile.write(line)

## 3. EXTRA: More complex example (using CSV module)

The approaches shown above works for many CSV files. However, some are a bit more complex. An example is the file `debate.csv` in the folder `../Data/csv_data`. This file contains transcripts of the 2016 (vice-)presidential debate from 26 September to 9 October. Let's have a look:

In [ ]:
# Read the file and print its content 
filename = "../Data/csv_data/debate.csv"
with open(filename, "r") as csvfile:
    content = csvfile.read()
    print(content[0:2000]) # only print first 2000 characters

This file contains 5 columns that are also separated by commas:

- Line
- Speaker
- Text
- Date

We can, however, notice two differences with the previous CSV file: this file contains a **header** and there are **double quotation marks** that are used to surround some of the cells. There is a good reason for these quotation marks. Some of the cells contain longer strings that have commas in them. These commas should be ignored when splitting the rows into columns. The quotation marks are thus a signal saying: "treat the part between quotation marks as one unit". Our previous approach does not account for this. Therefore, it will NOT work on this file, as illustrated below:

In [ ]:
# Read the file and print its content
filename = "../Data/csv_data/debate.csv"
with open(filename, "r") as csvfile:
    csv_data = []
    for row in csvfile:
        row = row.strip("\n")      # remove all newlines
        columns = row.split(",")   # split the line into columns
        csv_data.append(columns) 

# Print first 5 rows and their number of columns
for row in csv_data[:5]:
    print(f"This row is split into {len(row)} columns")
    print(row)
    print()

You may have ideas for a fix regarding this specific file and surely you will eventually find a good solution for it. However, for these more complex CSV files, we recommend that you make use of the **[csv module](https://docs.python.org/3/library/csv.html)**.

In [ ]:
import csv

We will focus on the following methods:
* **to read/write CSV/TSV with lists**: the methods `csv.reader()` and `csv.writer()`
* **to write CSV/TSV with dicts**: the methods `csv.DictReader()` and `csv.DictWriter()`

### 3.1 Reading and writing rows as lists
Below, you can find an example of how to use **`csv.reader()`** to convert the data to a list of lists. Note that the keyword arguments `delimiter` and `quotechar` further specify how the file should be read.

In [ ]:
filename = "../Data/csv_data/debate.csv"
with open(filename, "r") as csvfile:
    csv_data = []
    csv_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in csv_reader:
        csv_data.append(row)

In [ ]:
# Print only first 5 rows
for row in csv_data[:5]:
    print(row)

Writing a CSV file with the `csv` module works quite similar. In the code block below, we write our data back to a TSV file, using the `\t` as a delimiter and using double quotes as quote characters. `quoting=csv.QUOTE_MINIMAL` instructs the writer object to only quote those fields which contain special characters; see the [documentation](https://docs.python.org/3/library/csv.html) for other options. `writerows()` can be used to write multiple rows (list of lists); `writerow()` can be used to write a single row (a list). The code below uses both; `writerow()` for writing the header and `writerow()` for the rest of the rows.

In [ ]:
outfilename = "../Data/csv_data/debate.tsv"
with open(outfilename, "w") as outfile:
    header = ["Line", "Speaker", "Text", "Date"]
    csv_writer = csv.writer(outfile, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(header)
    csv_writer.writerows(csv_data)

### 3.1 Reading and writing rows as dicts
Below, you can find an example of how to use **`csv.DictReader`** to convert the data to a list of dicts using `csv.DictReader()`:

In [ ]:
filename = "../Data/csv_data/debate.csv"
with open(filename, "r") as csvfile:
    csv_data = []
    csv_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in csv_reader:
        csv_data.append(row)

You will notice that the data is represented as an `OrderedDict`. For now, you don't have to worry about the difference between a regular `dict` and an `OrderedDict`. Just remember that you can use them both in the same way.

In [ ]:
# Print only first 5 rows
for row in csv_data[:5]:
    print(row)

Accessing values in the `OrderedDict` works the same as with a regular `dict`:

In [ ]:
# Example: print all texts produced by the Audience
for row in csv_data:
    if row["Speaker"] == "Audience":
        print(row["Text"])

For writing a list of dicts, we use **`csv.DictWriter`**, which requires the `fieldnames` parameter specifying the header.  `writeheader()` is used to write a row with these specified fieldnames, as illustrated below:

In [ ]:
outfilename = "../Data/csv_data/debate.tsv"
with open(outfilename, "w") as outfile:
    header = csv_data[0].keys()       # use the keys of the first row as header
    csv_writer = csv.DictWriter(outfile, fieldnames=header, delimiter='\t', 
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writeheader()          # first write the header
    csv_writer.writerows(csv_data)    # then write the rows

## Exercises

### Exercise 1:
Have another look at the code below. Can you predict what would happen if you skip some of the steps? How will the data be different?

In [ ]:
filename = "../Data/baby_names/names_by_state/AK.TXT"
with open(filename, "r") as csvfile:
    csv_data = []
    for row in csvfile:
        #row = row.strip("\n")      # what would happen if you skip this step?
        columns = row.split(",")   
        csv_data.append(columns) 
print(csv_data[0:2])

In [ ]:
filename = "../Data/baby_names/names_by_state/AK.TXT"
with open(filename, "r") as csvfile:
    csv_data = []
    for row in csvfile:
        row = row.strip("\n")      
        #columns = row.split(",")   # what would happen if you skip this step?
        csv_data.append(row)        # replaced columns by row
print(csv_data[0:2])

### Exercise 2:
Read the csv data in `AK.TXT` and store it as a **list of lists**. Now print the following:
- all names that started with an M given in 1990
- all unique female names (hint: create a set)
- all names that were given more than 30 times in a certain year (print name+year)
- all unique names longer than 8 letters given between 1985 and 1990

Read the csv data in `AK.TXT` and store it as a **list of dicts**. Now print the following:
- all names that started with an M given in 1990
- all unique female names (hint: create a set)
- all names that were given more than 30 times in a certain year (print name+year)
- all unique names longer than 8 letters given between 1985 and 1990

### Exercise 3:
Can you think of a way to add a header to the TSV file below? Hint: make use of the dictionary keys.

In [ ]:
# Create list of dicts
address_book = [
    {"name":"John", "e-mail":"john@example.com", "phone":"555-1234"},
    {"name":"William", "e-mail":"william@example.com", "phone":"555-5678"},
    {"name":"Jane", "e-mail":"john@example.com", "phone":"555-7777"}
]

# Write the list of dicts to a TSV file
outfilename = "../Data/csv_data/address_book.tsv"
with open(outfilename, "w") as outfile:    
    
    # Write the header
    # your code here
    
    # Write the rows using the values of the dictionaries
    for row in address_book:
        column_values = row.values()
        line = "\t".join(column_values) + '\n'
        outfile.write(line)

### Exercise 4: 
Now *append* information about Jennifer and Justin (stored as a list) at the bottom of this `address_book.tsv` file (attention: make sure you do not overwrite the contents of the file).

In [ ]:
more_people = [
    ["Jennifer", "jennifer@example.com", "555-9876"],
    ["Justin", "justin@example.com", "555-5555"]
]

# Append the teacher data to the `address_book.tsv` file

### Exercise 5: 
Create a function `load_tabular_data(filename, delimiter)` that receives the filename and a delimiter as input parameters, and returns the file content as a list of lines. Then we can call this function for CSV (with a comma as an argument) and for TSV (with a tabulator as an argument). Also make sure that `delimiter` is a keyword parameter, with a defaul value ','.

In [ ]:
# Create your function here

    
# Now let's test the function
csv_filename = "../Data/baby_names/names_by_state/AK.TXT"
